In [ ]:
!pip install wandb

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import wandb
from wandb.keras import WandbMetricsLogger
import matplotlib.pyplot as plt

# get your data
(x_train, y_train), (x_val, y_val) = keras.datasets.cifar100.load_data()

# constant values
NUM_CLASSES = 100 #100 prediction classes
INPUT_SHAPE = (32,32,3) #shape of the input image 32x32 with 3 channels

# hyperparameters you will be tuning
BATCH_SIZE = 50 
EPOCHS = 5
LEARNING_RATE = 1e-3
L1NF = 16
FDROPOUT = 0.5

wandb.init(project = 'Tool Lab 3',
          config={
              'bs':BATCH_SIZE,
              'lr':LEARNING_RATE,
              'epochs': EPOCHS,
              'l1nf':L1NF,
              'fdr':FDROPOUT
          },
           name='RUN 0: L1NF 16 ' #this is your run name, please number your runs 0-n and tell what you changed
          )

# onehot encode your labels so your model knows its a category
y_train = tf.one_hot(y_train,
                     depth=y_train.max() + 1,
                     dtype=tf.float64)
y_val = tf.one_hot(y_val,
                   depth=y_val.max() + 1,
                   dtype=tf.float64)
  
y_train = tf.squeeze(y_train)
y_val = tf.squeeze(y_val)

# here is a basic model that you will add to
model = tf.keras.models.Sequential([
                  
                  # CHANGE THESE: these are layers you should mix up and change
                  layers.Conv2D(L1NF, (3, 3), input_shape = INPUT_SHAPE,
                                activation='relu',
                                padding='same'),
                  layers.MaxPooling2D(2,2),
                  layers.Dropout(FDROPOUT),
                  
                  # DO NOT CHANGE THESE. They should be at the end of your model
                  layers.Flatten(),
                  layers.Dense(NUM_CLASSES, activation='softmax')])


# feel free to experiment with this
model.compile(loss='categorical_crossentropy',
             optimizer=keras.optimizers.RMSprop(learning_rate=LEARNING_RATE),
             # DO NOT CHANGE THE METRIC. This is what you will be judging your model on
             metrics=['accuracy'],)


# here you train the model using some of your hyperparameters and send the data
# to weights and biases after every batch            
history = model.fit(x_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[WandbMetricsLogger(log_freq='batch')])